In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import akshare as ak
from tqdm import tqdm

import seaborn as sns

import statsmodels.graphics.tsaplots as sgt
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")
sns.set()
%matplotlib inline

In [51]:
df = ak.fund_etf_spot_em()

In [52]:
df.sort_values(by='总市值',ascending=False).head(10)

,代码,名称,最新价,涨跌额,涨跌幅,成交量,成交额,开盘价,最高价,最低价,昨收,换手率,流通市值,总市值
237,511880,银华日利ETF,100.921,0.003,0.00,1814928.0,1.831601e+10,100.918,100.923,100.913,100.918,15.00,122112795264,122112795264
301,511990,华宝添益ETF,99.994,-0.001,0.00,1410329.0,1.410208e+10,99.992,99.996,99.985,99.995,12.05,117051546086,117051546086
381,510300,沪深300ETF,3.942,-0.007,-0.18,3538923.0,1.396571e+09,3.941,3.959,3.936,3.949,1.94,71805846256,71805846256
222,588000,科创50ETF,1.101,0.001,0.09,26289318.0,2.883261e+09,1.098,1.104,1.090,1.100,4.26,67919223644,67919223644
520,510050,上证50ETF,2.581,-0.012,-0.46,3313576.0,8.569644e+08,2.589,2.598,2.576,2.593,1.56,54868618887,54868618887
315,510500,中证500ETF,6.198,-0.003,-0.05,762272.0,4.724450e+08,6.203,6.216,6.180,6.201,0.94,50185012027,50185012027
193,159915,创业板ETF,2.210,0.003,0.14,5795831.0,1.280742e+09,2.203,2.220,2.199,2.207,3.39,37829575214,37829575214
626,513050,中概互联网ETF,1.046,-0.008,-0.76,7568820.0,7.959014e+08,1.058,1.067,1.043,1.054,2.12,37368802742,37368802742
456,512880,证券ETF,0.895,-0.003,-0.33,8612430.0,7.729054e+08,0.897,0.901,0.895,0.898,2.32,33271885796,33271885796
601,513330,恒生互联网ETF,0.439,-0.003,-0.68,39431261.0,1.740454e+09,0.444,0.449,0.437,0.442,5.54,31273253917,31273253917


In [54]:
df_300 = ak.fund_etf_hist_em(symbol='510300',adjust='hfq')
df_500 = ak.fund_etf_hist_em(symbol='510500',adjust='hfq')
df_50 = ak.fund_etf_hist_em(symbol='510050',adjust='hfq')
df_kc50 = ak.fund_etf_hist_em(symbol='588000',adjust='hfq')

In [55]:
def report(ret,freqcy):
    ret.fillna(0,inplace=True)
    net = (1+ret).cumprod()
    yld_rate =  (net.iloc[-1]/net.iloc[0]) ** (252/freqcy/len(net)) - 1 # 年化收益率
    sigma =  np.std(ret) * np.sqrt(252/freqcy) # 年化波动率
    drawdown = net/net.cummax() - 1 #回撤
    maxdrawdown = max(-drawdown)
    sharpe = yld_rate / sigma # 夏普比 sharpe ratio
    summary_table = pd.DataFrame(index=[0])
    summary_table["年化收益率"] = str(round(yld_rate*100,2))+'%'
    summary_table["年化波动率"] = str(round(sigma*100,2))+'%'
    summary_table["风险收益比"] = str(round(sharpe,2))
    summary_table["最大回撤"] = str(round(maxdrawdown*100,2))+'%'
    summary_table["收益回撤比"] = str(round(yld_rate/ maxdrawdown,2))
    summary_table["月度胜率"] = str(round(( len(  ret[  ret>0])/len(  ret))*100 ,2))+'%'    
    summary_table.index = ['回测结果统计']
    return summary_table

def arima_rolling_str(df:pd.DataFrame,threshold,cost=3/10000):
    df.set_index(pd.to_datetime(df['日期']),inplace=True)
    df['收益率'] = df['收盘'].pct_change()
    df = df.loc['20180101':]
    df['pred'] = 0
    df = df[['收盘','收益率','pred']]
    for i in tqdm(range(len(df)-60)):
        model = SARIMAX(df['收益率'][i:i+60],order=(2,0,2))
        model_result = model.fit(disp=False)
        df.iloc[i+60,2] = model_result.forecast().values[0]
    df = df.iloc[60:]
    df.loc[df['pred']>threshold,'signal']=1
    df['signal'].fillna(0,inplace=True)
    df['stra_ret'] = df['signal']*df['收益率']
    df.loc[df['signal'].diff()!=0,'cost'] = cost
    df['cost'].fillna(0,inplace=True)
    df['stra_ret'] = df['stra_ret'] - df['cost']
    df['netvalue'] = (1+df['stra_ret']).cumprod()
    
    return df, report(df['stra_ret'],freqcy=1)

def arima_ma_rolling_str(df:pd.DataFrame,threshold,cost=3/10000):
    df.set_index(pd.to_datetime(df['日期']),inplace=True)
    df['收益率'] = df['收盘'].pct_change()
    df['ma20'] = df['收盘'].rolling(20).mean()
    df['ma60'] = df['收盘'].rolling(60).mean()
    df = df.loc['20180101':]
    df['pred'] = 0
    df = df[['收盘','收益率','ma20','ma60','pred']]
    for i in tqdm(range(len(df)-60)):
        model = SARIMAX(df['收益率'][i:i+60],order=(2,0,2))
        model_result = model.fit(disp=False)
        df.iloc[i+60,4] = model_result.forecast().values[0]
    df = df.iloc[60:]
    df.loc[df['ma20']>df['ma60'],'signal']=1
    df.loc[(df['pred']>threshold)&(df['ma20']<df['ma60']),'signal']=1
    df['signal'].fillna(0,inplace=True)
    df['stra_ret'] = df['signal']*df['收益率']
    df.loc[df['signal'].diff().fillna(0)!=0,'cost'] = cost
    df['cost'].fillna(0,inplace=True)
    df['stra_ret'] = df['stra_ret'] - df['cost']
    df['netvalue'] = (1+df['stra_ret']).cumprod()
    
    return df, report(df['stra_ret'],freqcy=1)

In [56]:
arima_rolling_str(df_300,3/10000)

100%|███████████████████████████████████████| 1267/1267 [00:40<00:00, 31.26it/s]


(               收盘       收益率      pred  signal  stra_ret    cost  netvalue
 日期                                                                       
 2018-04-03  4.126 -0.006023 -0.005217     0.0 -0.000300  0.0003  0.999700
 2018-04-04  4.116 -0.002424 -0.002066     0.0 -0.000000  0.0000  0.999700
 2018-04-09  4.121  0.001215 -0.000935     0.0  0.000000  0.0000  0.999700
 2018-04-10  4.196  0.018199  0.000066     0.0  0.000000  0.0000  0.999700
 2018-04-11  4.201  0.001192  0.003076     1.0  0.000892  0.0003  1.000591
 ...           ...       ...       ...     ...       ...     ...       ...
 2023-06-14  4.477  0.000000  0.001261     1.0  0.000000  0.0000  0.713077
 2023-06-15  4.542  0.014519 -0.000533     0.0 -0.000300  0.0003  0.712863
 2023-06-16  4.579  0.008146  0.007287     1.0  0.007846  0.0003  0.718456
 2023-06-19  4.549 -0.006552 -0.004088     0.0 -0.000300  0.0003  0.718241
 2023-06-20  4.542 -0.001539  0.001241     1.0 -0.001839  0.0003  0.716920
 
 [1267 rows x 7 columns

In [57]:
arima_rolling_str(df_500,3/10000)

100%|███████████████████████████████████████| 1267/1267 [00:41<00:00, 30.17it/s]


(               收盘       收益率      pred  signal  stra_ret    cost  netvalue
 日期                                                                       
 2018-04-03  1.787 -0.007773 -0.007516     0.0 -0.000300  0.0003  0.999700
 2018-04-04  1.772 -0.008394  0.005687     1.0 -0.008694  0.0003  0.991009
 2018-04-09  1.775  0.001693  0.008413     1.0  0.001693  0.0000  0.992686
 2018-04-10  1.788  0.007324 -0.010257     0.0 -0.000300  0.0003  0.992389
 2018-04-11  1.792  0.002237 -0.011236     0.0  0.000000  0.0000  0.992389
 ...           ...       ...       ...     ...       ...     ...       ...
 2023-06-14  1.962  0.001020 -0.000736     0.0  0.000000  0.0000  0.651452
 2023-06-15  1.980  0.009174  0.001246     1.0  0.008874  0.0003  0.657233
 2023-06-16  1.994  0.007071  0.000440     1.0  0.007071  0.0000  0.661880
 2023-06-19  1.991 -0.001505  0.002587     1.0 -0.001505  0.0000  0.660885
 2023-06-20  1.990 -0.000502  0.001465     1.0 -0.000502  0.0000  0.660553
 
 [1267 rows x 7 columns

In [58]:
arima_rolling_str(df_50,3/10000)

100%|███████████████████████████████████████| 1267/1267 [00:41<00:00, 30.82it/s]


(               收盘       收益率      pred  signal  stra_ret    cost  netvalue
 日期                                                                       
 2018-04-03  3.091 -0.002903 -0.004323     0.0 -0.000300  0.0003  0.999700
 2018-04-04  3.092  0.000324 -0.007465     0.0  0.000000  0.0000  0.999700
 2018-04-09  3.109  0.005498 -0.001336     0.0  0.000000  0.0000  0.999700
 2018-04-10  3.173  0.020585  0.009342     1.0  0.020285  0.0003  1.019979
 2018-04-11  3.173  0.000000  0.015408     1.0  0.000000  0.0000  1.019979
 ...           ...       ...       ...     ...       ...     ...       ...
 2023-06-14  3.186 -0.000314  0.000517     1.0 -0.000614  0.0003  0.838622
 2023-06-15  3.225  0.012241  0.000603     1.0  0.012241  0.0000  0.848888
 2023-06-16  3.248  0.007132  0.002729     1.0  0.007132  0.0000  0.854942
 2023-06-19  3.216 -0.009852 -0.000031     0.0 -0.000300  0.0003  0.854686
 2023-06-20  3.204 -0.003731 -0.001019     0.0 -0.000000  0.0000  0.854686
 
 [1267 rows x 7 columns

In [59]:
arima_rolling_str(df_kc50,3/10000)

100%|█████████████████████████████████████████| 571/571 [00:19<00:00, 29.41it/s]


(               收盘       收益率      pred  signal  stra_ret    cost  netvalue
 日期                                                                       
 2021-02-09  1.472  0.032982  0.002334     1.0  0.032682  0.0003  1.032682
 2021-02-10  1.478  0.004076  0.012178     1.0  0.004076  0.0000  1.036892
 2021-02-18  1.476 -0.001353  0.008945     1.0 -0.001353  0.0000  1.035489
 2021-02-19  1.472 -0.002710 -0.012345     0.0 -0.000300  0.0003  1.035178
 2021-02-22  1.433 -0.026495  0.005829     1.0 -0.026795  0.0003  1.007441
 ...           ...       ...       ...     ...       ...     ...       ...
 2023-06-14  1.080 -0.003690  0.000611     1.0 -0.003990  0.0003  0.758152
 2023-06-15  1.080  0.000000  0.005112     1.0  0.000000  0.0000  0.758152
 2023-06-16  1.098  0.016667  0.005258     1.0  0.016667  0.0000  0.770788
 2023-06-19  1.100  0.001821 -0.001205     0.0 -0.000300  0.0003  0.770557
 2023-06-20  1.101  0.000909  0.000441     1.0  0.000609  0.0003  0.771026
 
 [571 rows x 7 columns]

In [60]:
arima_ma_rolling_str(df_300,0.004)

100%|███████████████████████████████████████| 1267/1267 [00:40<00:00, 31.38it/s]


(               收盘       收益率     ma20      ma60      pred  signal  stra_ret  \
 日期                                                                           
 2018-04-03  4.126 -0.006023  4.26300  4.373200 -0.005217     0.0 -0.000000   
 2018-04-04  4.116 -0.002424  4.25340  4.368800 -0.002066     0.0 -0.000000   
 2018-04-09  4.121  0.001215  4.24200  4.364267 -0.000935     0.0  0.000000   
 2018-04-10  4.196  0.018199  4.23285  4.360717  0.000066     0.0  0.000000   
 2018-04-11  4.201  0.001192  4.22335  4.356967  0.003076     0.0  0.000000   
 ...           ...       ...      ...       ...       ...     ...       ...   
 2023-06-14  4.477  0.000000  4.45920  4.573033  0.001261     0.0  0.000000   
 2023-06-15  4.542  0.014519  4.45880  4.572833 -0.000533     0.0  0.000000   
 2023-06-16  4.579  0.008146  4.46055  4.573617  0.007287     1.0  0.007846   
 2023-06-19  4.549 -0.006552  4.45980  4.573183 -0.004088     0.0 -0.000300   
 2023-06-20  4.542 -0.001539  4.46155  4.572333  0.0

In [61]:
arima_ma_rolling_str(df_300,0.003)

100%|███████████████████████████████████████| 1267/1267 [00:40<00:00, 31.34it/s]


(               收盘       收益率     ma20      ma60      pred  signal  stra_ret  \
 日期                                                                           
 2018-04-03  4.126 -0.006023  4.26300  4.373200 -0.005217     0.0 -0.000000   
 2018-04-04  4.116 -0.002424  4.25340  4.368800 -0.002066     0.0 -0.000000   
 2018-04-09  4.121  0.001215  4.24200  4.364267 -0.000935     0.0  0.000000   
 2018-04-10  4.196  0.018199  4.23285  4.360717  0.000066     0.0  0.000000   
 2018-04-11  4.201  0.001192  4.22335  4.356967  0.003076     1.0  0.000892   
 ...           ...       ...      ...       ...       ...     ...       ...   
 2023-06-14  4.477  0.000000  4.45920  4.573033  0.001261     0.0  0.000000   
 2023-06-15  4.542  0.014519  4.45880  4.572833 -0.000533     0.0  0.000000   
 2023-06-16  4.579  0.008146  4.46055  4.573617  0.007287     1.0  0.007846   
 2023-06-19  4.549 -0.006552  4.45980  4.573183 -0.004088     0.0 -0.000300   
 2023-06-20  4.542 -0.001539  4.46155  4.572333  0.0

In [62]:
arima_ma_rolling_str(df_300,0.005)

100%|███████████████████████████████████████| 1267/1267 [00:40<00:00, 31.56it/s]


(               收盘       收益率     ma20      ma60      pred  signal  stra_ret  \
 日期                                                                           
 2018-04-03  4.126 -0.006023  4.26300  4.373200 -0.005217     0.0 -0.000000   
 2018-04-04  4.116 -0.002424  4.25340  4.368800 -0.002066     0.0 -0.000000   
 2018-04-09  4.121  0.001215  4.24200  4.364267 -0.000935     0.0  0.000000   
 2018-04-10  4.196  0.018199  4.23285  4.360717  0.000066     0.0  0.000000   
 2018-04-11  4.201  0.001192  4.22335  4.356967  0.003076     0.0  0.000000   
 ...           ...       ...      ...       ...       ...     ...       ...   
 2023-06-14  4.477  0.000000  4.45920  4.573033  0.001261     0.0  0.000000   
 2023-06-15  4.542  0.014519  4.45880  4.572833 -0.000533     0.0  0.000000   
 2023-06-16  4.579  0.008146  4.46055  4.573617  0.007287     1.0  0.007846   
 2023-06-19  4.549 -0.006552  4.45980  4.573183 -0.004088     0.0 -0.000300   
 2023-06-20  4.542 -0.001539  4.46155  4.572333  0.0

In [63]:
arima_ma_rolling_str(df_300,0.002)

100%|███████████████████████████████████████| 1267/1267 [00:40<00:00, 31.63it/s]


(               收盘       收益率     ma20      ma60      pred  signal  stra_ret  \
 日期                                                                           
 2018-04-03  4.126 -0.006023  4.26300  4.373200 -0.005217     0.0 -0.000000   
 2018-04-04  4.116 -0.002424  4.25340  4.368800 -0.002066     0.0 -0.000000   
 2018-04-09  4.121  0.001215  4.24200  4.364267 -0.000935     0.0  0.000000   
 2018-04-10  4.196  0.018199  4.23285  4.360717  0.000066     0.0  0.000000   
 2018-04-11  4.201  0.001192  4.22335  4.356967  0.003076     1.0  0.000892   
 ...           ...       ...      ...       ...       ...     ...       ...   
 2023-06-14  4.477  0.000000  4.45920  4.573033  0.001261     0.0  0.000000   
 2023-06-15  4.542  0.014519  4.45880  4.572833 -0.000533     0.0  0.000000   
 2023-06-16  4.579  0.008146  4.46055  4.573617  0.007287     1.0  0.007846   
 2023-06-19  4.549 -0.006552  4.45980  4.573183 -0.004088     0.0 -0.000300   
 2023-06-20  4.542 -0.001539  4.46155  4.572333  0.0